In [104]:
import json
import os
import pandas as pd
import numpy as np
import soundfile as sf
from pydub import AudioSegment
from num2words import num2words
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

## data exploration and preprocessing

In [4]:
data = pd.read_csv('numbers2/train.csv', sep=',')

In [5]:
data.head()

,path,gender,number
0,train/e332b996d3.wav,female,157105.0
1,train/e25afda49a.wav,female,374554.0
2,train/364f147340.wav,male,688694.0
3,train/5e0954b206.wav,female,265381.0
4,train/7130a67690.wav,male,955415.0


In [6]:
data['path'] = 'numbers2/' + data['path']

In [7]:
AudioSegment.from_wav(data['path'][200])

In [8]:
data['path'][200]

'numbers2/train/99dee92960.wav'

In [9]:
!ffmpeg -i numbers2/train/99dee92960.wav

ffmpeg version 4.2.4-1ubuntu0.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.3.0-10ubuntu2)
  configuration: --prefix=/usr --extra-version=1ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-l

In [10]:
labeled_data = data.iloc[:3000]

In [11]:
# labeled_data['number'] = labeled_data['number'].apply(lambda x: str(x)[:-2])

In [12]:
labeled_data.head()

,path,gender,number
0,numbers2/train/e332b996d3.wav,female,157105.0
1,numbers2/train/e25afda49a.wav,female,374554.0
2,numbers2/train/364f147340.wav,male,688694.0
3,numbers2/train/5e0954b206.wav,female,265381.0
4,numbers2/train/7130a67690.wav,male,955415.0


In [13]:
labeled_data['str_number'] = labeled_data['number'].apply(lambda x: num2words(int(x), lang='ru'))

/tmp/ipykernel_12979/2227669768.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_data['str_number'] = labeled_data['number'].apply(lambda x: num2words(int(x), lang='ru'))


In [14]:
labeled_data

,path,gender,number,str_number
0,numbers2/train/e332b996d3.wav,female,157105.0,сто пятьдесят семь тысяч сто пять
1,numbers2/train/e25afda49a.wav,female,374554.0,триста семьдесят четыре тысячи пятьсот пятьдес...
2,numbers2/train/364f147340.wav,male,688694.0,шестьсот восемьдесят восемь тысяч шестьсот дев...
3,numbers2/train/5e0954b206.wav,female,265381.0,двести шестьдесят пять тысяч триста восемьдеся...
4,numbers2/train/7130a67690.wav,male,955415.0,девятьсот пятьдесят пять тысяч четыреста пятна...
...,...,...,...,...
2995,numbers2/train/1c7676d4ec.wav,female,315305.0,триста пятнадцать тысяч триста пять
2996,numbers2/train/07d31d4913.wav,female,726822.0,семьсот двадцать шесть тысяч восемьсот двадцат...
2997,numbers2/train/db3268309f.wav,male,872553.0,восемьсот семьдесят две тысячи пятьсот пятьдес...
2998,numbers2/train/32a71e0375.wav,female,754212.0,семьсот пятьдесят четыре тысячи двести двенадцать


In [15]:
dict_array = []
labeled_data['arrays'] = labeled_data['str_number'].apply(lambda x: x.split(' '))
for i in labeled_data['arrays']:
    for j in i:
        if j not in dict_array:
            dict_array.append(j)

/tmp/ipykernel_12979/4087606259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_data['arrays'] = labeled_data['str_number'].apply(lambda x: x.split(' '))


In [16]:
text_numbers = []
for i in range(1000001):
    text = num2words(i, lang='ru')
    text = text.split(' ')
    for j in text:
        text_numbers.append(j)

In [17]:
text_numbers = set(text_numbers)

In [18]:
### The following labels are missing from the training set
### We need to generate samples with these words
set(text_numbers) - set(dict_array)

{'миллион', 'ноль'}

In [19]:
labeled_data.head()

,path,gender,number,str_number,arrays
0,numbers2/train/e332b996d3.wav,female,157105.0,сто пятьдесят семь тысяч сто пять,"[сто, пятьдесят, семь, тысяч, сто, пять]"
1,numbers2/train/e25afda49a.wav,female,374554.0,триста семьдесят четыре тысячи пятьсот пятьдес...,"[триста, семьдесят, четыре, тысячи, пятьсот, п..."
2,numbers2/train/364f147340.wav,male,688694.0,шестьсот восемьдесят восемь тысяч шестьсот дев...,"[шестьсот, восемьдесят, восемь, тысяч, шестьсо..."
3,numbers2/train/5e0954b206.wav,female,265381.0,двести шестьдесят пять тысяч триста восемьдеся...,"[двести, шестьдесят, пять, тысяч, триста, восе..."
4,numbers2/train/7130a67690.wav,male,955415.0,девятьсот пятьдесят пять тысяч четыреста пятна...,"[девятьсот, пятьдесят, пять, тысяч, четыреста,..."


In [20]:
def convert_and_getDur(x):
    sound = AudioSegment.from_file(x)
    sound = sound.set_frame_rate(16000)
    sound = sound.set_channels(1)
    sound.export(x, format="wav")
    return len(sound)/1000

In [21]:
labeled_data['duration'] = labeled_data['path'].apply(convert_and_getDur)

/tmp/ipykernel_12979/594879392.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_data['duration'] = labeled_data['path'].apply(convert_and_getDur)


In [22]:
labeled_data.drop(['gender', 'number', 'arrays'], axis=1, inplace=True)

/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [23]:
labeled_data = labeled_data.reindex(columns=['path', 
                                             'duration', 
                                             'str_number']).rename(columns={'str_number':'text',
                                                                            'path': 'audio_filepath'})

In [24]:
labeled_data.head()

,audio_filepath,duration,text
0,numbers2/train/e332b996d3.wav,2.372,сто пятьдесят семь тысяч сто пять
1,numbers2/train/e25afda49a.wav,3.361,триста семьдесят четыре тысячи пятьсот пятьдес...
2,numbers2/train/364f147340.wav,3.534,шестьсот восемьдесят восемь тысяч шестьсот дев...
3,numbers2/train/5e0954b206.wav,3.358,двести шестьдесят пять тысяч триста восемьдеся...
4,numbers2/train/7130a67690.wav,3.040,девятьсот пятьдесят пять тысяч четыреста пятна...


In [30]:
# train_manifest_file = 'train_data.json'
# with open(train_manifest_file, 'w') as fout:
#     for i in range(len(labeled_data)):
#         metadata = {
#                     "audio_filepath": labeled_data['audio_filepath'][i],
#                     "duration": labeled_data['duration'][i],
#                     "text": labeled_data['text'][i]
#                 }
#         json.dump(metadata, fout)
#         fout.write('\n')

## Generation and augmentation of data

### To generate data, we will use the Silero
#### Here I used an internal speech synthesis model with missing words

In [52]:
data_synth = pd.DataFrame(data = data_synth)
data_synth.head()

,audio_filepath,text
0,synthesis/synthesis_0.wav,ноль миллион два триста.
1,synthesis/synthesis_1.wav,ноль миллион триста два.
2,synthesis/synthesis_2.wav,ноль два миллион триста.
3,synthesis/synthesis_3.wav,ноль два триста миллион.
4,synthesis/synthesis_4.wav,ноль триста миллион два.


In [55]:
AudioSegment.from_wav(data_synth['audio_filepath'][2])

In [58]:
data_synth['duration'] = data_synth['audio_filepath'].apply(convert_and_getDur)
data_synth = data_synth.reindex(columns=['audio_filepath', 
                                             'duration', 
                                             'text'])

In [66]:
data_synth['text'] = data_synth['text'].apply(lambda x: x[:-1])
data_synth.head()

,audio_filepath,duration,text
0,synthesis/synthesis_0.wav,1.821,ноль миллион два триста
1,synthesis/synthesis_1.wav,1.809,ноль миллион триста два
2,synthesis/synthesis_2.wav,1.844,ноль два миллион триста
3,synthesis/synthesis_3.wav,1.763,ноль два триста миллион
4,synthesis/synthesis_4.wav,1.774,ноль триста миллион два


In [72]:
data_with_missing_words = pd.concat([labeled_data, data_synth]).reset_index(drop=True)

In [75]:
data_with_missing_words.head()

,audio_filepath,duration,text
0,numbers2/train/e332b996d3.wav,2.372,сто пятьдесят семь тысяч сто пять
1,numbers2/train/e25afda49a.wav,3.361,триста семьдесят четыре тысячи пятьсот пятьдес...
2,numbers2/train/364f147340.wav,3.534,шестьсот восемьдесят восемь тысяч шестьсот дев...
3,numbers2/train/5e0954b206.wav,3.358,двести шестьдесят пять тысяч триста восемьдеся...
4,numbers2/train/7130a67690.wav,3.040,девятьсот пятьдесят пять тысяч четыреста пятна...


In [136]:
list(text_numbers)

['четырнадцать',
 'десять',
 'семьдесят',
 'шестнадцать',
 'два',
 'тысяча',
 'три',
 'миллион',
 'семьсот',
 'восемнадцать',
 'восемьдесят',
 'одна',
 'двадцать',
 'сто',
 'ноль',
 'пятнадцать',
 'пятьсот',
 'сорок',
 'восемьсот',
 'девятьсот',
 'тридцать',
 'тринадцать',
 'девяносто',
 'две',
 'один',
 'семнадцать',
 'шестьдесят',
 'шесть',
 'тысяч',
 'пятьдесят',
 'тысячи',
 'двести',
 'семь',
 'пять',
 'четыре',
 'триста',
 'двенадцать',
 'четыреста',
 'девять',
 'одиннадцать',
 'шестьсот',
 'девятнадцать',
 'восемь']

In [103]:
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
])

In [108]:
### Can make more audio files
#os.makedirs('aug/')
def get_augmentation(x):
    new_name = 'aug/' + x[0].split('/')[-1][:-4] + '_AUG.wav'
    au, sr = sf.read(x[0])
    augmented_samples = augment(samples=au, sample_rate=sr)
    sf.write(new_name, augmented_samples, sr)
    durtaion = augmented_samples.size / sr
    return new_name, durtaion, x[1]

aug_data = data_with_missing_words[['audio_filepath', 'text']].apply(lambda x: get_augmentation(x), axis=1)

/opt/conda/lib/python3.8/site-packages/audiomentations/core/transforms_interface.py:57: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/audiomentations/core/transforms_interface.py:57: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(


In [117]:
aug_data = pd.DataFrame.from_records(data=aug_data, columns=['audio_filepath', 'duration', 'text'])

In [120]:
data_with_missing_words_aug = pd.concat([data_with_missing_words, aug_data]).reset_index(drop=True)

In [123]:
os.makedirs('csvs/')
labeled_data.to_csv('csvs/labeled_data.csv', sep='|', index=False)
data_synth.to_csv('csvs/data_synth.csv', sep='|', index=False)
aug_data.to_csv('csvs/aug_data.csv', sep='|', index=False)
data_with_missing_words.to_csv('csvs/data_with_missing_words.csv', sep='|', index=False)
data_with_missing_words_aug.to_csv('csvs/data_with_missing_words_aug.csv', sep='|', index=False)

In [128]:
data_with_missing_words_aug.shape

(6048, 3)

In [130]:
data_with_missing_words_aug = data_with_missing_words_aug.sample(frac=1).reset_index(drop=True)

In [132]:
train_data = data_with_missing_words_aug.iloc[: 5100]
valid_data = data_with_missing_words_aug.iloc[5100 : 5600].reset_index(drop=True)
test_data = data_with_missing_words_aug.iloc[5600 : ].reset_index(drop=True)

In [135]:
os.makedirs('json_data/')

train_manifest_file = 'json_data/train_data.json'
with open(train_manifest_file, 'w') as fout:
    for i in range(len(train_data)):
        metadata = {
                    "audio_filepath": train_data['audio_filepath'][i],
                    "duration": train_data['duration'][i],
                    "text": train_data['text'][i]
                }
        json.dump(metadata, fout)
        fout.write('\n')
        
valid_manifest_file = 'json_data/valid_data.json'
with open(valid_manifest_file, 'w') as fout:
    for i in range(len(valid_data)):
        metadata = {
                    "audio_filepath": valid_data['audio_filepath'][i],
                    "duration": valid_data['duration'][i],
                    "text": valid_data['text'][i]
                }
        json.dump(metadata, fout)
        fout.write('\n')
        
test_manifest_file = 'json_data/test_data.json'
with open(test_manifest_file, 'w') as fout:
    for i in range(len(test_data)):
        metadata = {
                    "audio_filepath": test_data['audio_filepath'][i],
                    "duration": test_data['duration'][i],
                    "text": test_data['text'][i]
                }
        json.dump(metadata, fout)
        fout.write('\n')